In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from keras.callbacks import EarlyStopping
from keras.layers import ConvLSTM2D


In [1]:
# load the dataset
dataframe =pd.read_csv('10_min_total_ml_LSTM.csv', delimiter=';', usecols=[1])

print(dataframe)


In [ ]:
print('Raw data size:------------>',dataframe.shape)
dataframe = dataframe.dropna()
print('Dataset size after dropna():',dataframe.shape)


Raw data size:------------> (52506, 1)
Dataset size after dropna(): (52506, 1)


In [2]:
for col_name in dataframe.keys():
  dataframe = dataframe[~dataframe[col_name].astype(str).str.contains(r"[a-z]")]
  dataframe = dataframe.astype({col_name: float})


original_dataframe_size=dataframe.shape
print('Original dataframe size:------>',original_dataframe_size)
print(dataframe.head(5))

In [3]:
# dataframe=np.array(dataframe)
# plt.hist(dataframe)

In [ ]:
# Outlier Removal 2
import scipy.stats as stats
z = np.abs(stats.zscore(dataframe))
dataframe= dataframe[(z<1.1).all(axis=1)]
# print(dataframe)
print(dataframe.shape)

(33056, 1)


In [ ]:
#Convert pandas dataframe to numpy array
# dataset = dataframe.values
plt.plot(dataframe)
dataset = dataframe

In [5]:
#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1)) #Also try QuantileTransformer
dataset = scaler.fit_transform(dataset)
plt.plot(dataset)

In [ ]:
#We cannot use random way of splitting dataset into train and test as
#the sequence of events is important for time series.
#first 66% values: traindatset, remaining: test dataset
# split into train and test sets:
train_size = int(len(dataset) * 0.66)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]



#seq_size is the number of previous time steps to use as 
#input variables to predict the next time period.

def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
        #print(i)
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])
        
    return np.array(x),np.array(y)
    

seq_size = 600 # Number of time steps to look back 
#Larger sequences (look further back) may improve forecasting.

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)

print("Shape of training set: {}".format(trainX.shape))
print("Shape of test set: {}".format(testX.shape))

# print(trainX)
# print(trainY)

Shape of training set: (21755, 60)
Shape of test set: (11179, 60)


In [ ]:
testY.shape

(1, 11179)

* Select one of the time series prediction model and continue

## **Single LSTM Model**

In [ ]:
# Single LSTM
Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

print('Single LSTM with hidden Dense...')
model = Sequential()
model.add(LSTM(40, input_shape=(None, seq_size)))
model.add(Dense(30))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, 
#                        verbose=1, mode='auto', restore_best_weights=True)
model.summary()
print('Train...')





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 100)              44400     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 44,501
Trainable params: 44,501
Non-trainable params: 0
_________________________________________________________________
Train...


### **Bidirectional LSTM Model**

In [ ]:
#Bidirectional LSTM
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#For some sequence forecasting problems we may need LSTM to learn
# sequence in both forward and backward directions
from keras.layers import Bidirectional
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(None, seq_size)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
print('Train...')

## **Stacked LSTM Model**

In [ ]:
# #Stacked LSTM with 1 hidden dense layer
# # reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# model = Sequential()
# model.add(LSTM(40, activation='relu', return_sequences=True, input_shape=(None, seq_size)))
# model.add(LSTM(40, activation='relu'))
# model.add(Dense(35))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mean_squared_error')

# model.summary()
# print('Train...')

## **Convolutional LSTM Model**

In [ ]:
#ConvLSTM
#The layer expects input as a sequence of two-dimensional images, 
#therefore the shape of input data must be: [samples, timesteps, rows, columns, features]

trainX = trainX.reshape((trainX.shape[0], 1, 1, 1, seq_size))
testX = testX.reshape((testX.shape[0], 1, 1, 1, seq_size))

model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu', input_shape=(1, 1, 1, seq_size)))
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
#print('Train...')

* While you selected and run the code for the particular prediction-> fit the model

# Fit the Model

In [ ]:

model.fit(trainX, trainY, validation_data=(testX, testY),
          verbose=0, epochs=50)


# make predictions

In [ ]:
# make predictions

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions back to prescaled values
#This is to compare with original input values
#SInce we used minmaxscaler we can now use scaler.inverse_transform
#to invert the transformation.
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
print(trainY.shape)
print(testPredict.shape)
# print(testPredict)

(1, 21755)
(11179, 1)


In [ ]:
from scipy.stats import pearsonr



# calculate root mean squared error on train and test
trainScore1 =  math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore1))
print('Train Pearson correlation Coefficient (R) is:', pearsonr(trainY[0].ravel(), trainPredict[:,0].ravel()))

testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
print('Test Pearson correlation Coefficient (R) is:', pearsonr(testY[0].ravel(), testPredict[:,0].ravel()))

Train Score: 0.51 RMSE
Train Pearson correlation Coefficient (R) is: (0.9542918812182102, 0.0)
Test Score: 0.39 RMSE
Test Pearson correlation Coefficient (R) is: (0.9719353717219992, 0.0)


# **Plot Results**

In [6]:
# Plot original data, Train data, and Test data in the same Graph

trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[seq_size:len(trainPredict)+seq_size, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(seq_size*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset), label='Original Data')
plt.plot(trainPredictPlot, label='Train Data', color='chartreuse')
plt.plot(testPredictPlot, label='Test Data')
plt.xlabel('Data Pionts',fontsize=10)
plt.ylabel('Motor load (Mwh)',fontsize=10)
plt.legend(loc='lower left')
plt.rcParams["figure.figsize"] = (17,10)
# plt.rcParams["figure.autolayout"] = True
ax = plt.gca()
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)
# ax.set_facecolor("cornsilk")
# plt.rcParams['font.size'] = 10
plt.show()


In [7]:
# Plot train and test data in the same Graph
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)